In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

TRAIN DATA 

In [2]:
movies_train_df = pd.read_csv(Path('movies.csv'))

In [3]:
movies_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7930 entries, 0 to 7929
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   name      7930 non-null   object 
 1   rating    7853 non-null   object 
 2   genre     7930 non-null   object 
 3   year      7930 non-null   int64  
 4   released  7928 non-null   object 
 5   score     7927 non-null   float64
 6   votes     7665 non-null   float64
 7   director  7930 non-null   object 
 8   writer    7927 non-null   object 
 9   star      7929 non-null   object 
 10  country   7665 non-null   object 
 11  budget    5497 non-null   float64
 12  gross     7741 non-null   object 
 13  company   7913 non-null   object 
 14  runtime   7926 non-null   float64
dtypes: float64(4), int64(1), object(10)
memory usage: 929.4+ KB


In [4]:
movies_train_df['label']=''
for index,row in movies_train_df.iterrows():
    if (row['score']<=5):
        movies_train_df.loc[index,'label']=0
    elif (row['score']>5) & (row['score']<=7):
        movies_train_df.loc[index, 'label']=1
    elif (row['score']>7):
         movies_train_df.loc[index, 'label']=2
    else:
        movies_train_df.loc[index, 'label']=0

In [5]:
movies_train_df['label'].value_counts()

1    5180
2    2037
0     713
Name: label, dtype: int64

In [6]:
# movies_train_df['score'] = movies_train_df['score'].replace(np.nan, 0)

In [7]:
movies_train_df['label'].value_counts()

1    5180
2    2037
0     713
Name: label, dtype: int64

In [8]:
movies_train_df = movies_train_df.drop('budget', axis = 1)

In [9]:
movies_train_df = movies_train_df.dropna(axis = 0, how='any')
movies_train_df

,name,rating,genre,year,released,score,votes,director,writer,star,country,gross,company,runtime,label
0,The Shining,R,Drama,1980,13-Jun-80,8.4,927000.0,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,46998772,Warner Bros.,146.0,2
1,The Blue Lagoon,R,Adventure,1980,2-Jul-80,5.8,65000.0,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,58853106,Columbia Pictures,104.0,1
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,20-Jun-80,8.7,1200000.0,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,538375067,Lucasfilm,124.0,2
3,Airplane!,PG,Comedy,1980,2-Jul-80,7.7,221000.0,Jim Abrahams,Jim Abrahams,Robert Hays,United States,83453539,Paramount Pictures,88.0,2
4,Caddyshack,R,Comedy,1980,25-Jul-80,7.3,108000.0,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,39846344,Orion Pictures,98.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7651,The Call of the Wild,PG,Adventure,2020,21-Feb-20,6.8,42000.0,Chris Sanders,Michael Green,Harrison Ford,Canada,111105497,20th Century Studios,100.0,1
7652,The Eight Hundred,Not Rated,Action,2020,28-Aug-20,6.8,3700.0,Hu Guan,Hu Guan,Zhi-zhong Huang,China,461421559,Beijing Diqi Yinxiang Entertainment,149.0,1
7653,The Quarry,R,Crime,2020,17-Apr-20,5.4,2400.0,Scott Teems,Scott Teems,Shea Whigham,United States,3661,Prowess Pictures,98.0,1
7656,Tulsa,PG-13,Comedy,2020,3-Jun-20,5.0,294.0,Scott Pryor,Scott Pryor,Scott Pryor,United States,413378,Pryor Entertainment,120.0,0


In [10]:
y = movies_train_df['label'].astype('int')
X = movies_train_df.drop(['label', 'score'], axis = 1)

In [11]:
y.value_counts()

1    4896
2    1922
0     594
Name: label, dtype: int64

In [12]:
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()
transformed = lb_make.fit_transform(X['name'])
X['name'] = transformed

transformed = lb_make.fit_transform(X['rating'])
X['rating'] = transformed

transformed = lb_make.fit_transform(X['genre'])
X['genre'] = transformed

transformed = lb_make.fit_transform(X['director'])
X['director'] = transformed

transformed = lb_make.fit_transform(X['writer'])
X['writer'] = transformed

transformed = lb_make.fit_transform(X['star'])
X['star'] = transformed

transformed = lb_make.fit_transform(X['country'])
X['country'] = transformed

transformed = lb_make.fit_transform(X['company'])
X['company'] = transformed

transformed = lb_make.fit_transform(X['released'])
X['released'] = transformed

X

,name,rating,genre,year,released,votes,director,writer,star,country,gross,company,runtime
0,6372,6,6,1980,434,927000.0,2471,3851,979,52,46998772,2179,146.0
1,5373,6,1,1980,1084,65000.0,2169,1564,306,53,58853106,686,104.0
2,4956,4,0,1980,1182,1200000.0,1063,2464,1642,53,538375067,1446,124.0
3,277,4,4,1980,1084,221000.0,1248,1920,2112,53,83453539,1697,88.0
4,993,6,4,1980,1672,108000.0,1010,499,387,53,39846344,1665,98.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7651,5432,4,1,2020,1264,42000.0,438,2838,895,6,111105497,17,100.0
7652,5586,3,0,2020,1917,3700.0,1048,1602,2645,8,461421559,350,149.0
7653,6265,6,5,2020,765,2400.0,2404,3726,2300,53,3661,1786,98.0
7656,6792,5,4,2020,2119,294.0,2399,3719,2241,53,413378,1788,120.0


In [13]:
y

0       2
1       1
2       2
3       2
4       2
       ..
7651    1
7652    1
7653    1
7656    0
7659    0
Name: label, Length: 7412, dtype: int32

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [15]:
y_train

2709    1
1922    1
7003    1
3404    1
1641    1
       ..
994     1
5404    1
4185    2
257     1
5369    1
Name: label, Length: 5559, dtype: int32

In [16]:
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_train_scaled

array([[-0.57018375,  0.69423572,  0.54519736, ..., -0.23782861,
        -0.98818259,  1.08394537],
       [-0.55347165,  0.69423572,  0.54519736, ..., -0.46517883,
         0.65110402, -0.28976709],
       [-1.66363264,  0.69423572,  0.16353861, ..., -0.42528305,
         1.32984746, -0.50956108],
       ...,
       [-0.47086612, -0.17457481,  0.92685612, ..., -0.32091671,
        -0.24277685, -0.12492159],
       [ 0.20860016,  0.69423572,  0.92685612, ..., -0.44779464,
         1.07531867,  1.13889387],
       [-1.47120474, -0.17457481,  0.16353861, ..., -0.06947432,
        -1.32906936, -0.45461258]])

In [17]:
classifier = LogisticRegression()
classifier

LogisticRegression()

In [18]:
y_train

2709    1
1922    1
7003    1
3404    1
1641    1
       ..
994     1
5404    1
4185    2
257     1
5369    1
Name: label, Length: 5559, dtype: int32

In [19]:
classifier.fit(X_train, y_train)

LogisticRegression()

In [20]:
y_predict = classifier.predict(X_test)

In [21]:
y_predict

array([1, 1, 1, ..., 1, 1, 2])

In [22]:
print(y_test - y_predict)

3079    0
5147    0
2376    0
5008    0
2737    1
       ..
6177    0
1765    1
1836    0
1288    1
5679    0
Name: label, Length: 1853, dtype: int32


In [23]:
classifier.score(X_train, y_train)

0.7015650296815974

In [24]:
classifier.score(X_test, y_test)

0.712358337830545

In [25]:
classifier.score(X_test, y_predict)

1.0

In [26]:
y_predict = pd.DataFrame(classifier.predict(X_test), index=X_test.index, columns=['Predictions'])
y_predict_df = y_predict.merge(y_test, how = 'outer', left_index = True, right_index = True)
y_predict_df

,Predictions,label
3079,1,1
5147,1,1
2376,1,1
5008,1,1
2737,1,2
...,...,...
6177,1,1
1765,1,2
1836,1,1
1288,1,2


TEST DATA START

In [41]:
# save csv so we don't have to run the loop again
y_predict_df.to_csv("predictions_rating.csv", index=False)

In [27]:
movies_test_df = movies_test_df = pd.read_csv(Path('upcoming_df_2022.csv'))
movies_test_df

,movie_id,title,movie_rating,release_date,runtime,genre_name,Director,Writer,Actors,Language,Poster,metascore,imdb_rating
0,0,Studio 666,R,2/25/2022,0,"Comedy, Horror, Music",BJ McDonnell,"Dave Grohl, Jeff Buhler, Rebecca Hughes","Jenna Ortega, Will Forte, Leslie Grossman",English,https://m.media-amazon.com/images/M/MV5BMmU5Nj...,0,0.0
1,1,Butter,0,1/31/2022,19,"Short, Comedy, Drama",Anita Lee,Madeleine Withington,"Amber Adams, Matthew Backer, Jenna Bell",English,0,0,0.0
2,2,The Batman,PG-13,3/4/2022,175,"Action, Crime, Drama",Matt Reeves,"Matt Reeves, Peter Craig, Bob Kane","Barry Keoghan, Zoë Kravitz, Robert Pattinson",English,https://m.media-amazon.com/images/M/MV5BYTExZT...,0,0.0
3,4,He Was Asking for It: A Noir Film,0,4/30/2022,0,"Short, Crime",Rocky Karlage,"Rocky Karlage, Staci Layne Wilson","Rocky Karlage, Patrick Lamba, Charlie Jiin",English,https://m.media-amazon.com/images/M/MV5BMjA5NG...,0,0.0
4,5,X,R,3/18/2022,0,Horror,Ti West,Ti West,"Jenna Ortega, Mia Goth, Brittany Snow",English,https://m.media-amazon.com/images/M/MV5BYjI1OW...,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,75,Strange World,0,11/23/2022,0,"Animation, Action, Adventure",Don Hall,Qui Nguyen,Alan Tudyk,English,https://m.media-amazon.com/images/M/MV5BMDNjOT...,0,0.0
71,76,Avatar 2,0,12/16/2022,0,"Action, Adventure, Sci-Fi",James Cameron,"James Cameron, Josh Friedman","Kate Winslet, Zoe Saldana, Michelle Yeoh",English,https://m.media-amazon.com/images/M/MV5BNmM1Nm...,0,0.0
72,77,Aquaman and the Lost Kingdom,0,12/16/2022,0,"Action, Adventure, Fantasy",James Wan,"David Leslie Johnson-McGoldrick, Paul Norris, ...","Temuera Morrison, Jason Momoa, Nicole Kidman",English,https://m.media-amazon.com/images/M/MV5BM2QyYT...,0,0.0
73,78,Untitled Illumination Entertainment Project,0,12/21/2022,0,"Animation, Adventure, Comedy","Aaron Horvath, Michael Jelenic","Matthew Fogel, Shigeru Miyamoto","Anya Taylor-Joy, Chris Pratt, Seth Rogen",English,https://m.media-amazon.com/images/M/MV5BODMzNm...,0,0.0


In [28]:
movies_test_df['label']=''
for index,row in movies_test_df.iterrows():
    if (row['imdb_rating']<=5):
        movies_test_df.loc[index,'label']=0
    elif (row['imdb_rating']>5) & (row['imdb_rating']<=7):
        movies_test_df.loc[index, 'label']=1
    elif (row['imdb_rating']>7):
         movies_test_df.loc[index, 'label']=2

In [29]:
movies_test_df['label'].value_counts()

0    68
1     5
2     2
Name: label, dtype: int64

In [30]:
movies_test_df = movies_test_df.dropna(axis = 0, how='any')
movies_test_df

,movie_id,title,movie_rating,release_date,runtime,genre_name,Director,Writer,Actors,Language,Poster,metascore,imdb_rating,label
0,0,Studio 666,R,2/25/2022,0,"Comedy, Horror, Music",BJ McDonnell,"Dave Grohl, Jeff Buhler, Rebecca Hughes","Jenna Ortega, Will Forte, Leslie Grossman",English,https://m.media-amazon.com/images/M/MV5BMmU5Nj...,0,0.0,0
1,1,Butter,0,1/31/2022,19,"Short, Comedy, Drama",Anita Lee,Madeleine Withington,"Amber Adams, Matthew Backer, Jenna Bell",English,0,0,0.0,0
2,2,The Batman,PG-13,3/4/2022,175,"Action, Crime, Drama",Matt Reeves,"Matt Reeves, Peter Craig, Bob Kane","Barry Keoghan, Zoë Kravitz, Robert Pattinson",English,https://m.media-amazon.com/images/M/MV5BYTExZT...,0,0.0,0
3,4,He Was Asking for It: A Noir Film,0,4/30/2022,0,"Short, Crime",Rocky Karlage,"Rocky Karlage, Staci Layne Wilson","Rocky Karlage, Patrick Lamba, Charlie Jiin",English,https://m.media-amazon.com/images/M/MV5BMjA5NG...,0,0.0,0
4,5,X,R,3/18/2022,0,Horror,Ti West,Ti West,"Jenna Ortega, Mia Goth, Brittany Snow",English,https://m.media-amazon.com/images/M/MV5BYjI1OW...,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,75,Strange World,0,11/23/2022,0,"Animation, Action, Adventure",Don Hall,Qui Nguyen,Alan Tudyk,English,https://m.media-amazon.com/images/M/MV5BMDNjOT...,0,0.0,0
71,76,Avatar 2,0,12/16/2022,0,"Action, Adventure, Sci-Fi",James Cameron,"James Cameron, Josh Friedman","Kate Winslet, Zoe Saldana, Michelle Yeoh",English,https://m.media-amazon.com/images/M/MV5BNmM1Nm...,0,0.0,0
72,77,Aquaman and the Lost Kingdom,0,12/16/2022,0,"Action, Adventure, Fantasy",James Wan,"David Leslie Johnson-McGoldrick, Paul Norris, ...","Temuera Morrison, Jason Momoa, Nicole Kidman",English,https://m.media-amazon.com/images/M/MV5BM2QyYT...,0,0.0,0
73,78,Untitled Illumination Entertainment Project,0,12/21/2022,0,"Animation, Adventure, Comedy","Aaron Horvath, Michael Jelenic","Matthew Fogel, Shigeru Miyamoto","Anya Taylor-Joy, Chris Pratt, Seth Rogen",English,https://m.media-amazon.com/images/M/MV5BODMzNm...,0,0.0,0


In [31]:
movies_test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75 entries, 0 to 74
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   movie_id      75 non-null     int64  
 1   title         75 non-null     object 
 2   movie_rating  75 non-null     object 
 3   release_date  75 non-null     object 
 4   runtime       75 non-null     int64  
 5   genre_name    75 non-null     object 
 6   Director      75 non-null     object 
 7   Writer        75 non-null     object 
 8   Actors        75 non-null     object 
 9   Language      75 non-null     object 
 10  Poster        75 non-null     object 
 11  metascore     75 non-null     int64  
 12  imdb_rating   75 non-null     float64
 13  label         75 non-null     object 
dtypes: float64(1), int64(3), object(10)
memory usage: 8.8+ KB


In [32]:
y = movies_test_df['label'].astype('int')
X = movies_test_df.drop('label', axis = 1)
# X.info()

In [33]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75 entries, 0 to 74
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   movie_id      75 non-null     int64  
 1   title         75 non-null     object 
 2   movie_rating  75 non-null     object 
 3   release_date  75 non-null     object 
 4   runtime       75 non-null     int64  
 5   genre_name    75 non-null     object 
 6   Director      75 non-null     object 
 7   Writer        75 non-null     object 
 8   Actors        75 non-null     object 
 9   Language      75 non-null     object 
 10  Poster        75 non-null     object 
 11  metascore     75 non-null     int64  
 12  imdb_rating   75 non-null     float64
dtypes: float64(1), int64(3), object(9)
memory usage: 8.2+ KB


In [34]:
y.value_counts()

0    68
1     5
2     2
Name: label, dtype: int64

In [35]:
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()
transformed = lb_make.fit_transform(X['title'])
X['title'] = transformed

transformed = lb_make.fit_transform(X['movie_rating'])
X['movie_rating'] = transformed

transformed = lb_make.fit_transform(X['genre_name'])
X['genre_name'] = transformed

transformed = lb_make.fit_transform(X['Director'])
X['Director'] = transformed

transformed = lb_make.fit_transform(X['Writer'])
X['Writer'] = transformed

transformed = lb_make.fit_transform(X['Actors'])
X['Actors'] = transformed

transformed = lb_make.fit_transform(X['Language'])
X['Language'] = transformed

transformed = lb_make.fit_transform(X['Poster'])
X['Poster'] = transformed

transformed = lb_make.fit_transform(X['release_date'])
X['release_date'] = transformed

X

,movie_id,title,movie_rating,release_date,runtime,genre_name,Director,Writer,Actors,Language,Poster,metascore,imdb_rating
0,0,45,3,16,0,18,7,17,35,1,19,0,0.0
1,1,11,0,2,19,34,5,49,3,1,0,0,0.0
2,2,47,2,19,175,7,48,51,8,1,45,0,0.0
3,4,25,0,26,0,35,60,67,59,1,13,0,0.0
4,5,73,3,17,0,32,73,71,34,1,50,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,75,44,0,11,0,14,20,63,1,1,2,0,0.0
71,76,5,0,13,0,3,31,34,43,1,32,0,0.0
72,77,3,0,13,0,2,32,18,68,1,1,0,0.0
73,78,71,0,14,0,16,0,53,5,1,34,0,0.0


In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [37]:
classifier = LogisticRegression()
classifier

LogisticRegression()

In [38]:
classifier.fit(X_train, y_train)

C:\Users\mgian\anaconda3\envs\PythonData\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [39]:
classifier.predict

<bound method LinearClassifierMixin.predict of LogisticRegression()>

In [54]:
y_predict = classifier.predict(X_test)
y_predict

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0])

In [55]:
classifier.score(X_train, y_train)

1.0

In [56]:
classifier.score(X_test, y_test)

0.8947368421052632

In [57]:
classifier.score(X_test, y_predict)

1.0